# Homework 6

## Problem 1: RSA Encryption/Decryption

### 1(a): Encryption

Given a modulus $N = pq$ (for unknown $p$ and $q$ distinct primes), encryption exponent $e$, and message $m \in \{2, \ldots , N-1\}$, your function should output the encrypted message (to be sent).

Note that $m \in \mathbb{Z}$, but your encrypted message should be in $\mathbb{Z}/N\mathbb{Z}$.

Also, you *have* to use Sage's fast exponentiation modulo $N$ (i.e., perform powers in $\mathbb{Z}/N\mathbb{Z}$), as otherwise it is too slow.

(This is really very simple!)

In [ ]:
def rsa_encr(N, e, m):
    """
    Given the modulus N, the encryption expoenent e, and a message m, uses RSA
    to encrypt the message.

    INPUTS:
    N: the modulus (a product of two distinct primes),
    e: the encryption exponent (a positive integer).
    m: the message (positive integer).

    OUTPUT:
    The encrypted message (an integer modulo N).
    """
    ...

### 1(b): Decryption

Here we will assume, for the decryption, that all the ingredients are known, so the input is simply an encoded message $x$ in $\mathbb{Z}/N\mathbb{Z}$ (so not an integer!) and the decryption exponent $d$.  It should return the decrypted message *as an integer* (and *not* in $\mathbb{Z}/N\mathbb{Z}$).

Again, do all powers in $\mathbb{Z}/N\mathbb{Z}$!

In [ ]:
def rsa_decr(x, d):
    """
    Given a message x encrypted by the RSA and the decoding expoent d, gives
    back the original message.

    INPUTS:
    x: the encrypted message (an integer modulo N).
    d: the decryption exponent.

    RETURN:
    The decrypted message (a positive integer).
    """
    ...

### 1(c): Generating the Keys

Given $b$, your function should generate two (*distinct!*) random $b$-bit primes ($p$ and $q$), the encryption exponent $e$, which for us will *always* be $e = 2^{16}+1$ (which is prime), and the corresponding decryption exponent $d$.

**Note:** Since we are choosing $e = 2^{16}+1$, we have to make sure our primes $p$ and $q$ are such that $\gcd(e,(p-1)(q-1)) = 1$!  If not, we must choose other primes.  (But note that $2^{16}+1$ is itself prime, so what we need is that $e \nmid (p-1)$ and $e \nmid (q-1)$.)

The output has to be a pair of lists `v` and `w`, with $v = (p, q, d)$ (for decryption) and $w = (pq, e)$.  (So, $v$ is private and $w$ is public.)

Note that $d$ must be an integer between $1$ and $(p-1)(q-1)$.  (You can always reduce it modulo $(p-1)(q-1)$!)

**Hints:** 
1) Use Sage's `random_prime`.
2) Make sure to test that $p$ does not divide $p-1$ nor $q-1$.
3) Make sure $p \neq q$.

In [ ]:
def rsa_keys(b):
    """
    Given b, returns (p, q, d), (p*q, e), where p and q are two distinct random
    b-bit primes, and e and d are the encryption/decryption exponent for RSA.

    INPUT:
    b: number of bits for the primes (positive integer).

    OUTPUTS:
    (p, q, d): p and q are two b-bit primes and d is a decryption exponent.
    (p*q, e): e is the encryption exponent, so d*e is 1 modulo p*q.
    """
    e = 2^16 + 1
    ...

We will test those, but keep in mind that if even only one of the functions above is incorrect, the whole test might be compromised!  (I will check it though!)

In [ ]:
number_of_tests = 20
minb = 20
maxb = 30


def test_key(b, v, w):
    p, q, d = v
    N, e = w
    if N != p * q:
        print("N must be p*q.")
        return False
    if p == q:
        print("p and q must be different.")
        return False
    if ((p - 1) % e == 0) or ((q - 1) % e == 0):
        print("e cannot divide p or q")
        return False
    if not (is_prime(p) and is_prime(q)):
        print("p and q must be primes.")
        return False
    if p < 2 ^ (b - 1) or q < 2 ^ (b - 1):
        print("p or q too small.")
        return False
    if p >= 2 ^ b or q >= 2 ^ b:
        print("p or q too large.")
        return False
    phi = (p - 1) * (q - 1)
    if Mod(d, phi) * Mod(e, phi) != Mod(1, phi):
        print("d not the invese of e.")
        return False
    if d >= phi or d < 0:
        print("d is too large or too small.")
        return False
    return True


for _ in range(number_of_tests):
    b = randint(minb, maxb + 1)
    [p, q, d], [N, e] = rsa_keys(b)
    if not test_key(b, [p, q, d], [N, e]):
        print("The rsa_keys function failed!")
        break

    v = [p, q, d]
    w = [N, e]
    m = randint(2, N)
    decr_message = rsa_decr(rsa_encr(N, e, m), d)
    if decr_message != m:
        print(
            f"Fails for {v = }, {w = }, {m = }.  Decryption gave {decr_message}."
        )
        break
else:
    print("It works!")

## Problem 2: Carmichael Numbers

In this problem you will test if a number greater than or equal to $2$ is composite or either prime or a [Carmichael number](https://luisfinotti.org/pcimc/11-Primality.html#def-carmichael) by using [Fermat's Little Theorem](https://luisfinotti.org/pcimc/11-Primality.html#th-flt-2).

The output, for a given $n$ to be tested, is either

- an *integer* that is either $a \in \{2, \ldots , n-1\}$ which is a witness for $n$ (if composite),
- or $-1$ if $a$ is either prime or a Carmichael Number.

**Important:** You cannot use any Sage function related to primality!  (You can use `Mod`, though, to work in $\mathbb{Z}/n\mathbb{Z}$.)

Remember that a *witness* for the compositeness of $n$ is simply some $a \in \{1, 2, \ldots, n-1\}$ such that $a^n \not\equiv a \pmod{n}$.

**Note:** Use exponentiation in $\mathbb{Z}/n\mathbb{Z}$, not in $\mathbb{Z}$!  (Points will be taken if you don't, as it slows down the algorithm immensely!)

In [ ]:
def carm_test(n):
    """
    Test if n is a Carmichael number.  Returns -1 if n is either prime or
    Carmichael number, otherwise it should return a witness.

    INPUT:
    n: an integer greater than 1 to be tested.

    OUTPUT:
    A witness for the compositeness of n or -1 if n is either a prime or
    a Carmichael number.
    """
    if n == 2:
        return -1
    ...

Let's test it (against my list of Carmichael numbers -- this can take a minute):

In [ ]:
max_to_try = 10000
carm_nos = {
    561,
    1105,
    1729,
    2465,
    2821,
    6601,
    8911,
}  # my set of Carmichael numbers up to 10,000

for n in range(2, max_to_try):
    result = carm_test(n)
    if result == -1:
        if (not is_prime(n)) and (n not in carm_nos):
            print(
                f"{n} is neither prime nor a Carmichael number, while your function says it is."
            )
            break
    else:
        if is_prime(n) or n in carm_nos:
            print(
                f"{n} is either prime or a Carmichael number, while your function says it is not."
            )
            break
        if Mod(result, n)^n == Mod(result, n):
            print(f"You gave {result} as a witness, but it is not.")
            break
else:
    print("It works!")

### Some Remarks (Optional)

Note that there are only seven Carmichael numbers between $2$ and $10{,}000$.  So, the probability our test gives that $n$ is either Carmichael number or prime (i.e., output $-1$), and it is not prime (and so a Carmichael number) for any $n$ in this range is: 

In [ ]:
n = prime_pi(10000)
7/(7 + n), 7.0/(7 + n)

So, less than 0.6% chance.

We can also see how fast we can find a witness for the non-primes/non-Carmichael numbers.  The routine below (*if your function is correct!*) should give you and idea of how small the first witness is.  The smallest it is, the quickest the test tells that he number is composite.

The output is an average of the size of first witness $a$ (output of the function) compared to (i.e., divided by) $n-1$ (the maximal possible output), for the first $10{,}000$ non-prime/non-Carmichael numbers:

In [ ]:
carm_nos = {
    561,
    1105,
    1729,
    2465,
    2821,
    6601,
    8911,
}  # my set of Carmichael numbers up to 10,000

sum_of_values = 0
total = 0

for n in range(2, 10000 + 1):
    if (not is_prime(n)) and (n not in carm_nos):
        witness = carm_test(n)
        if witness == -1:
            print("Something is wrong!")
            break
        sum_of_values += witness / (n - 1)
        total += 1

numerical_approx(sum_of_values / total)

The correct value is about 0.15%, so, on average, the first witness is about 0.15% for $n-1$.

## Problem 3: Miller-Rabin Test

In this problem you will implement the [Miller-Rabin algorithm](https://luisfinotti.org/pcimc/11-Primality.html#al-mr).  (See also the [example in the book](https://luisfinotti.org/pcimc/11-Primality.html#id1).)

Given $a$ and $n > 2$, two integers, your function should return $1$ if the test tells you that $a$ is a Miller-Rabin witness for the compositeness of $n$, and $-1$ otherwise (so the test would fail to determine if $n$ is composite).

**Notes:** 
- To determine $k$ such that $n-1 = 2^k \cdot q$, with $q$ odd, use Sage's `valuation(n - 1, 2)`.  (You definitely should not use `factor`, as it does a lot more than you need!)
- Make sure to work in $\mathbb{Z}/n\mathbb{Z}$.

In [ ]:
def mr(n, a):
    """
    Given n and a, checks if a is a Miller Rabin witness for n.  If so, it
    should return 1.  If not, it should return -1.

    INPUTS:
    n: an integer greater than 2 to test for compositeness.
    a: a possible Miller-Rabin witness.

    OUTPUT:
    1 if a is a Miller-Rabin witness, and -1 if not.
    """
    if n == 2:
        return -1
    ...

Test it:

In [ ]:
number_of_tests = 20
min_n = 20
max_n = 10^4

# test specific example
witness_25 = [2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23]

cont = True  # continue after first test?
for a in range(1, 25):
    if (mr(25, a) == 1) and (a not in witness_25):
        print(f"{a} should not be a witness for 25.")
        cont = False
        break
    if (mr(25, a) == -1) and (a in witness_25):
        print(f"{a} should be a witness for 25.")
        cont = False
        break

if cont:
    for _ in range(number_of_tests):
        # test composite numbers first
        n = ZZ(randint(min_n, max_n))
        while is_prime(n):
            n = ZZ(randint(min_n, max_n))
        a = 2
        while mr(n, a) == -1 and a < n:
            a += 1
        if a == n:
            print(f"{n} is composite, but your test fails!")
            break

        # now test for primes
        n = random_prime(max_n, lbound = min_n)
        a = 2
        while mr(n, a) == -1 and a < n:
            a += 1
        if a < n:
            print(f"{n} is prime, but your test says it's composite!")
            break
    else:
        print("It works!")

## Problem 4: Miller-Rabin Witness

As [stated in the book](https://luisfinotti.org/pcimc/11-Primality.html#th-mr_witness):

**Theorem:** If $n$ is odd and composite, at least $3/4$ of $a \in \{2, 3, 4, \ldots, n-1\}$ should be Miller-Rabin witnesses of $n$.

This was not proved, so we verify it numerically here.

### 4(a): Number of Witnesses

Given an odd composite number $n$ (you do not need to verify it), the output of your function should be the total number of Miller-Rabin witnesses (between $2$ and $n-1$) for that $n$.

**Note:** Use your Miller-Rabin function `mr` above.  If your Miller-Rabin function is incorrect or does not work, the tests below will fail, but I will check the correctness independently of the test results.  So, if the code for this part is correct, you can get full credit even if `mr` is incorrect.

In [ ]:
def mrws(n):
    """
    Given n, it gives how many Miller-Rabin witnesses n has.

    INPUT:
    n: an integer greater than 2.

    OUTPUT:
    Number of Miller-Rabin witnesess for n.
    """
    ...

Test it (for some few examples I've computed beforehand):

In [ ]:
list_n = [63, 195, 357, 753, 979 ]
list_count = [60, 192, 354, 750, 976]

for n, count in zip(list_n, list_count):
    res_count = mrws(n)
    if res_count != count:
        print(f"Your function gave {res_count} MR-witnesses for {n}, while mine gave {count}.")
        break
else:
    print("It seems to work!")

### 4(b): Average Proportion of Witnessess

Here you will find the average proportion if Miller-Rabin witnesses for all *odd* composite numbers from $9$ to (and including) $1007$.  In other words, you compute the average of `mrws(n)/(n-2)` for all **composite odd** numbers `n` between $9$ and $1007$.

**Hint:** I've done some averages for you in the problem about Carmichael numbers.  If you are not sure how to do it, maybe looking at it might help.

In [ ]:
...

According to my computation, the result should be `0.986696208654307`.

I've done it up to $10005$ (which would take a bit too long to ask you to do), and the average was `0.996304578975721`, so even higher.  The [theorem above](https://luisfinotti.org/pcimc/11-Primality.html#th-mr_witness) says that *at least* 75% of the $a$'s are MR-witnesses, but in practice it seems that a lot more than 75% are (in most cases).  The computation above says that the average is closer to 99.6%!

As [stated in the book](https://luisfinotti.org/pcimc/11-Primality.html#proportion-of-witnesses), with the 75% chance from the theorem, we can *guarantee* that if $n$ is odd composite and we pick 10 integers to try to find the witness, the probability that none would be a witness is *at most* one in a million.  But, in practice, it is *a lot* more unlikely!

## Problem 5: Probabilistic Primality Test

In this problem we will use the Miller-Rabin test to do a probabilisitc primality test.

Given $n$, a number to test if prime or not, and $k$ the number of witnesses to try, apply the Miller-Rabin test `mr(n,a)` for $k$ `a`'s chosen *randomly* in $\{2, \ldots, n-1\}$.  (**Hint:** You can use `randint`.)

If for any `a` we get `mr(n,a) = 1`, return `False` (as `n` is composite).  If you try `k` *different* (make sure to not repeat!) `a`'s, and for all of them get `mr(n,a) = -1`, return `True`, as `n` is *probably* prime (with a chance of only *at most* $1$ in $4^k$ that it is not).

**Hint:** To make sure you do not pick a repeated $a$ create an empty set, with `set_of_as = set()`, and create a while loop to get new $a$'s: while $a$ is in the set, choose a new $a$.  When you are done with the look, add this new $a$ to the set (with `set_of_as.add(a)`).

**Note:** Again, this will not work if your `mr` function is incorrect, but I will grade this part independent of `mr`.

In [ ]:
def mrpt(n, k):
    """
    Use at most k numbers with the Miller-Rabin test to see if n is probably
    prime or definitely composite.

    INPUTS:
    n: the number to test for compositeness (positive integer).
    k: number of tries with the Miller-Rabin test.

    OUTPUT:
    False if n is composite and True if n is likely prime.
    """
    set_of_as = set()
    ...

Test it!  (I will use a high `k`, and thus just check your function with `is_prime`.)

In [ ]:
number_of_tests = 30
k = 20
min_n = 10^3
max_n = 10^5

for _ in range(number_of_tests):
    # test composite numbers first
    n = ZZ(randint(min_n, max_n))
    while is_prime(n):
        n = ZZ(randint(min_n, max_n))
    if mrpt(n,k):
        print(f"{n} is composite, but your test says it's (likely) prime.")
        break
    # test primes
    n = random_prime(max_n, lbound = min_n)
    if not mrpt(n, k):
        print(f"{n} is prime, but your test says it's composite!") 
        break
else:
    print("It works!")

### Efficiency (Optional)

Let's test the efficiency of some algorithms to test primality.

First, let's define the "naive method": seach for a divisor less than or equal to $\lfloor \sqrt{n} \rfloor$.

In [ ]:
def naive_pt(n):
    """
    Naive primality test.  Given n, returns True if prime and False if
    composite.

    INPUT:
    n: positive integer to be tested for primality.

    OUTPUT:
    True if n is prime, False if it is composite.
    """
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    for d in range(3, floor(sqrt(n)) + 1, 2):  # only odds!
        if n % d == 0:
            return False  # found proper divisor
    return True

Let's fix a prime to test:

In [ ]:
p = next_prime(10^17)

First the naive (it might take a little while):

In [ ]:
%%time
naive_pt(p)

Now, we could try the Carmichael (probabilistic) test `carm_test`, but it is *very* slow.  (You can try it yourself, if you want, but be prepared to wait!)  

So, let's skip it and compare it to the Miller-Rabin, with `k=20`.

In [ ]:
%%time
mrpt(p, 20)

It's very fast!

We can go even farther:

In [ ]:
p = next_prime(10^30)

In [ ]:
%%time
mrpt(p, 20)

Indeed, *very* fast!